In [ ]:
''' This cell contains all constants thay may differ on our machines '''

ELASTIC_HOST = 'localhost'
ELASTIC_PORT = 9200
COLLECTION_DIRECTORY = "../byweb" # directory with .out files to process
COLLECTION_DIRECTORY_MYSTEM = "../byweb_stem" # directory with .out files after mystem processing

QUERIES_FILE = "web2008_adhoc.xml"
RELEVANCE_FILE = "relevant_table_2009.xml"

In [ ]:
%config IPCompleter.greedy=True
import re
import json
from collections import defaultdict
from tqdm import tqdm_notebook as tqdm
from elasticsearch import Elasticsearch
from elasticsearch.helpers import parallel_bulk
from pymystem3 import Mystem
from sklearn.feature_extraction.text import CountVectorizer
import requests
from time import time

In [ ]:
es = Elasticsearch([{'host': ELASTIC_HOST, 'port': ELASTIC_PORT, 'timeout': 360, 'maxsize': 25}])

In [ ]:
settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            }
        }
    }
}

In [ ]:
def recreate_index():
    es.indices.delete(index='hw2index')
    es.indices.create(index='hw2index', body=settings)

In [ ]:
recreate_index()

In [ ]:
def create_es_action(index, doc_id, document):
    return {
        '_index': index,
        '_id': doc_id,
        '_source': document
    }

In [ ]:
class Document:
    def __init__(self, doc_url, doc_id, sz_bytes, sz_words):
        self.url = doc_url       # document url
        self.id = doc_id         # unique document id (str)
        self.sz_bytes = sz_bytes # document size in bytes before deleting html markup
        self.sz_words = sz_words # number of words in document before deleting html markup
        self.words = []          # list of words in document after deleting html markup
        self.links = []          # lisk of links in document

In [ ]:
import os

In [ ]:
from tqdm import tqdm
from tqdm import tqdm_notebook
import pickle

class BaseDocumentProcessor:
    def process(self, document):
        pass
    def result(self):
        pass

def process_file(d, f, processor, pbar):
    print("processing", os.path.join(d, f))
    with open(os.path.join(d, f), "rb") as fin:
        while True:
            pbar.update(1)
            try:
                document = pickle.load(fin)
            except:
                break
            processor.process(document)

def process_collection(directory, processor):
    pbar = tqdm(total = 200000)
    for file in os.listdir(directory):
        if (file.endswith(".out")):
            process_file(directory, file, processor, pbar)

In [ ]:

class IndexDocs(BaseDocumentProcessor):
    def __init__(self):
        """ do all initialization here """
        self.actions = []
    
    def process(self, document):
        """ document: Document (see first cell)
            process each document here """
        #print(json.dumps({'content' : document.words}))
        self.actions.append(create_es_action('hw2index', document.id, json.dumps({'content' : document.words})))
        
    def result(self):
        return self.actions
        


In [ ]:
def es_actions_generator():
    processor = IndexDocs()        
    process_collection(COLLECTION_DIRECTORY, processor)
    return processor.result()

In [ ]:
import time

In [ ]:
start = time.time()
for ok, result in tqdm_notebook(parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)
end = time.time()
print('Time=' + str(end - start))


In [ ]:
import requests
param = (('v', ''),) # '-v' is for --verbose

# call the class's method to get an HTTP response model
resp = requests.get(f'http://{ELASTIC_HOST}:{ELASTIC_PORT}/_cat/indices', params=param)


In [ ]:
resp.text

In [ ]:
settings = {
    'mappings': {
        'properties': {
            'content': {
                'type': 'text'
            }
        }
    },
    "settings": {
        "analysis" : {
            "analyzer" : {
                "my_analyzer" : {
                    "tokenizer" : "standard",
                    "filter" : ["lowercase", "my_snow"]
                }
            },
            "filter" : {
                "my_snow" : {
                    "type" : "snowball",
                    "language" : "russian"
                }
            }
        }
    }
}

In [ ]:
recreate_index()

In [ ]:
start = time.time()
for ok, result in tqdm_notebook(parallel_bulk(es, es_actions_generator(), queue_size=4, thread_count=4, chunk_size=1000)):
    if not ok:
        print(result)
end = time.time()
print('Time=' + str(end - start))



In [ ]:
requests.get(f'http://{ELASTIC_HOST}:{ELASTIC_PORT}/_cat/indices', params=param).text

In [ ]:
from bs4 import BeautifulSoup

def read_queries():
    queries = {}
    with open(QUERIES_FILE, "rt", encoding="cp1251") as file:
        bs = BeautifulSoup(file.read())
        for task in bs.find_all("task"):
            queries[task["id"]] = task.text
    return queries

def read_relevance():
    relevance = {}
    with open(RELEVANCE_FILE, "rt", encoding="cp1251") as file:
        bs = BeautifulSoup(file.read())
        for task in bs.find_all("task"):
            rel = [doc["id"] for doc in task.find_all("document") if doc["relevance"] == "vital"]
            if rel:
                relevance[task["id"]] = rel
    return relevance

In [ ]:
relevance = read_relevance()
queries = read_queries()

In [ ]:
def build_query(query):
    return { 'query': { 'bool': { 'should': [ { 'match': { 'content': query } } ] } } }

def run_search(query, size):
    search_result = es.search(index='hw2index', body=build_query(query), size=size)['hits']
    return [hit['_id'] for hit in search_result['hits']]

def count_metrics():
    p = 0
    r = 0
    rp = 0
    ma_p = 0
    n = 0
    for qid, query in queries.items():
        if qid not in relevance.keys():
            continue
        n += 1
        rel = relevance[qid]
        relevant = len(rel)
        hits = run_search(query, max(20, relevant))
        hits_r = hits[:relevant]
        hits_20 = hits[:20]
        true_positive_20 = sum(1 for did in hits_20 if did in rel)
        true_positive_r = sum(1 for did in hits_r if did in rel)
        p += true_positive_20 / 20
        r += true_positive_20 / relevant
        rp += true_positive_r / relevant
        ma_p += sum(sum(1 for did in hits[:k] if did in rel) / k for k in range(1, 21)) / 20
    print("p@20:", p / n)
    print("r@20:", r / n)
    print("r-precision:", rp / n)
    print("map@20:", ma_p / n)

In [ ]:
count_metrics()

In [ ]:
class MystemProcessor(BaseDocumentProcessor):
    def __init__(self, fout):
        self.m = Mystem(grammar_info=False, disambiguation=False)
        reg = re.compile('[a-zа-яё0-9\-]')
        self.filterFunc = lambda w : reg.match(w)
        self.fout = fout

    def process(self, document):
        text = ' '.join(document.words).lower()
        lemmas = self.m.lemmatize(text)
        words = list(filter(self.filterFunc, lemmas))
        doc = Document(document.url, document.id, document.sz_bytes, document.sz_words)
        doc.words = words
        doc.links = document.links
        pickle.dump(doc, self.fout)

         
    def result(self):
        pass

def process_collection_with_mystem(directory, outdirectory):
    pbar = tqdm(total = 200000)
    for file in os.listdir(directory):
        if (file.endswith(".out")):        
            with open(os.path.join(outdirectory, file), "wb") as fout:
                processor = MystemProcessor(fout)
                process_file(directory, file, processor, pbar)

In [ ]:
process_collection_with_mystem(COLLECTION_DIRECTORY, COLLECTION_DIRECTORY_MYSTEM)

In [ ]:
def stem_queries(queries):
    m = Mystem(grammar_info=False, disambiguation=False)
    reg = re.compile('[a-zа-яё0-9\-]')
    filterFunc = lambda w : reg.match(w)
    result = {}
    for (qid, text) in queries:
        result[qid] = ' '.join(filter(filterFunc, m.lemmatize(text)))
    return result